In [1]:
#import necessary libraries
import numpy as np
import pandas as pd

In [2]:
#load the dataset
data=pd.read_excel('C:\\Users\\HP\\jupyter projects\\bookrecommend\\data.xlsx')

In [3]:
data.rename(columns = {'Rack No':'RackNo'}, inplace = True)

In [4]:
#How many times have we read which book category?
book_counts = pd.DataFrame(data["Category"].value_counts())

In [5]:
#Most read books
book_counts.sort_values("Category", ascending=False)

,Category
C Programming,30
Electronics,17
System Architecture,13
Networking,11
UNIX,9
Operating System,8
Software engineering,8
MATH,7
DAA,7
DMS,6


In [6]:
#count value for title of the book
book_counts = pd.DataFrame(data["Title"].value_counts())
#Most read books
book_counts.sort_values("Title", ascending=False)

,Title
Hadoop,2
Operating Systems edition one,1
PHP and MySQL Web Development,1
Advanced Programming in the UNIX Environment,1
System Software,1
...,...
Introduction to Algorithims,1
The Complete Reference C++,1
Foundations Of Software Testing by bichkar,1
Software Engineering,1


In [7]:
#We named the books with less than 100 reads as rare books.
rare_book = book_counts[book_counts["Title"] <= 100].index
#Number of rarely read books
rare_book.nunique()

184

In [8]:
data.groupby('Title')['Rating '].mean().sort_values(ascending=False).head(10)


Title
Computer Organization                                                   5.0
Computer Graphics                                                       5.0
Introduction to Operating Systems                                       5.0
Computer Organisation                                                   5.0
Foundations Of Software Testing by andrew                               5.0
Computer Programming in C                                               5.0
Introduction to Embedded System                                         5.0
Computer Sciecne Engineering: Software Engineering & web Technology.    5.0
Programming with C Data Structure                                       5.0
The Complete Reference  JAVA                                            5.0
Name: Rating , dtype: float64

In [9]:
data.groupby('Title')['Rating '].count().sort_values(ascending=False).head(10)

Title
Hadoop                                                          2
A Detailed Textbook of Engineering Physics                      1
Introduction to The Design and Analysis of Algorithms           1
Introduction to UNIX & SHELL Programming                        1
Itroduction to UNIX and SHELL Programming by venkateshmurthy    1
KEEPING AHEAD CONDUCED TECHNICAL GUIDES                         1
Learning PHP, My SQL and Java Script                            1
Lex Yacc                                                        1
MBA GUIDE TO MICROSOFT EXCEL 2002                               1
Machine Learning                                                1
Name: Rating , dtype: int64

In [10]:
ratings = pd.DataFrame(data.groupby('Title')['Rating '].mean())

In [11]:
ratings['rating_numbers'] = pd.DataFrame(data.groupby('Title')['Rating '].count())

In [12]:
ratings=ratings[ratings['rating_numbers'] >= 50]

In [13]:
data.drop_duplicates(['User id','Title'], inplace=True)

In [14]:
from scipy.sparse import csr_matrix
rating_pivot = data.pivot(index = 'Title',columns = 'User id', values = 'Rating ').fillna(0)
rating_matrix = csr_matrix(rating_pivot.values)

In [15]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [16]:
query_index = np.random.choice(rating_pivot.shape[0])
distances, indices = model_knn.kneighbors(rating_pivot.iloc[61, :].values.reshape(1, -1))

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Data amd Computer Communications :

1: Learning PHP, My SQL and Java Script , with distance of 1.0:
2: Itroduction to UNIX and SHELL Programming by venkateshmurthy, with distance of 1.0:
3: Lex Yacc , with distance of 1.0:
4: Machine Learning , with distance of 1.0:


In [17]:
def recommend(book_name):
    book_id = np.where(rating_pivot.index==book_name)[0][0]
    distances,suggestions=model_knn.kneighbors(rating_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors = 11)
    
    
    books=[]
    for i in range(len(suggestions)):
        if i==0:
            print("The suggestions for ",book_name,"are : ")
        if not i:
            books = rating_pivot.index[suggestions[i]]
    for i in range(1,len(books)):
         print(str(i) + ": " + books[i] )       
               
            
recommend('Communication Networks ')

The suggestions for  Communication Networks  are : 
1: Fundamentals of Computer''s
2: Computer Sciecne Engineering: Software Engineering & web Technology.
3: Modern Digital Signal Processing
4: KEEPING AHEAD CONDUCED TECHNICAL GUIDES
5: Lex Yacc 
6: Machine Learning 
7: Learning PHP, My SQL and Java Script 
8: Itroduction to UNIX and SHELL Programming by venkateshmurthy
9: Microcontroller (ARM) and Embedded Systems 
10: Introduction to UNIX & SHELL Programming 


In [18]:
def recommend(book_name):
    book_id = np.where(rating_pivot.index==book_name)[0][0]
    distances,suggestions=model_knn.kneighbors(rating_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors = 11)
    books=[]
    for i in range(len(suggestions)):
        if i==0:
            print("The suggestions for ",book_name,"are : ")
        if not i:
            books = rating_pivot.index[suggestions[i]]
    for i in range(1,len(books)):
            print(str(i) + ": " + books[i] )

    return books

In [19]:
recommend('Embedded Systems')

The suggestions for  Embedded Systems are : 
1: Foundations Of Software Testing by ben
2: Embedded Systems
3: Learning PHP, My SQL and Java Script 
4: Itroduction to UNIX and SHELL Programming by venkateshmurthy
5: Lex Yacc 
6: Machine Learning 
7: Introduction to UNIX & SHELL Programming 
8: KEEPING AHEAD CONDUCED TECHNICAL GUIDES
9: Introduction to The Design and Analysis of Algorithms 
10: MBA GUIDE TO MICROSOFT EXCEL 2002


Index(['Embedded System Design ', 'Foundations Of Software Testing by ben',
       'Embedded Systems', 'Learning PHP, My SQL and Java Script ',
       'Itroduction to UNIX and SHELL Programming by venkateshmurthy',
       'Lex Yacc ', 'Machine Learning ',
       'Introduction to UNIX & SHELL Programming ',
       'KEEPING AHEAD CONDUCED TECHNICAL GUIDES',
       'Introduction to The Design and Analysis of Algorithms ',
       'MBA GUIDE TO MICROSOFT EXCEL 2002'],
      dtype='object', name='Title')

In [20]:
s = data.apply(lambda x: pd.Series(x['Category']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Category'
gen_md = data.drop('Category', axis=1).join(s)
gen_md.head(3).transpose()

,0,1,2
User id,50.0,19.0,30.0
Title,A Detailed Textbook of Engineering Physics,Advanced Calculus and Numerical Methods [-],Advanced Computer Architecture one
Authorname,S.P. Basavaraju,Dr. K.S.C,Hai Hwang & Naresh jotwani
Edition,1st,1st,3rd
Call No,530 BAS,510.76 KSC,621.381958 HAI
IMPRINT,"Bangalore: Subhash Stores, 2018","Bangalore: Sudha Publication, 2019","New Delhi: Mc Graw Hill, 2016"
Accession Number,"20541, 20563, 20566, 20550, 20560, 20573, 2056...","19642, 19634, 19635, 19633, 19629, 19643, 1963...","18402, 18407, 18405, 18404, 18406, 18408, 1840..."
Bibliographic level,MONOGRAPH,MONOGRAPH,MONOGRAPH
Copies,40.0,15.0,8.0
RackNo,74.0,13.0,18.0


In [21]:
def build_chart(Authorname, percentile=0.85):
    df = gen_md[gen_md['Category'] == Authorname]
#     vote_counts = df[df['votecount'].notnull()]['votecount'].astype('int')
#     vote_averages = df[df['vote average'].notnull()]['vote average'].astype('int')
#     C = vote_averages.mean()
#     m = vote_counts.quantile(percentile)
    
    qualified = df[(df['votecount'] >= 100) & (df['votecount'].notnull()) & 
                   (df['vote average'].notnull())][['Title',  
                                                'votecount', 
                                                'vote average', 
                                                'Copies', 
                                                'Authorname','RackNo']]
    qualified['votecount'] = qualified['votecount'].astype('int')
    qualified['vote average'] = qualified['vote average'].astype('int')
    
    
    
    return qualified

In [22]:
a=build_chart('IOT')

In [23]:
a

,Title,votecount,vote average,Copies,Authorname,RackNo
110,Internet of Things by rajkamal,170,4,5.0,Raj Kamal,92.0
111,Internet of Things by arshdeep,232,3,4.0,Arshdeep Bahga & Vijay Madisetti,4.0
112,Internet of Things by srinivas,420,6,8.0,"Srinivasa K.G, Siddesha GM & Hanumantha Raju R",20.0
120,IOT Fundementals,373,4,8.0,"David Hanes, Ganzalo Salgneiro, Patrick Grosse...",75.0


In [24]:
import pickle
pickle.dump(gen_md,open('category.pkl','wb'))

In [25]:
book=pickle.load(open('category.pkl','rb'))

In [53]:
def build_chart(Authorname):
    df = book[book['Category'] == Authorname]
    qualified = df[(df['votecount'] >= 50) & (df['votecount'].notnull()) & 
                   (df['vote average'].notnull())][['Title',  
                                                'votecount', 
                                                'vote average', 
                                                'Copies', 
                                                'Authorname','RackNo']]
    qualified['votecount'] = qualified['votecount'].astype('int')
    qualified['vote average'] = qualified['vote average'].astype('int')
    
    
    
    return qualified

In [54]:
a=build_chart('MATH')

In [55]:
a

,Title,votecount,vote average,Copies,Authorname,RackNo
1,Advanced Calculus and Numerical Methods [-],156,3,15.0,Dr. K.S.C,13.0
3,Advanced Engineeering Mthematics,84,4,1.0,C Ray Wylie & Louis c Barrett,61.0
36,Computer Science Engineering: Engineering Math...,57,6,1.0,Gate Forum Centre,17.0
56,Discrete and Combinatorical Mathematics,92,10,5.0,Ralph P Grimaldi and Ramana B V,99.0
57,Discrete and Combinatrorial Mathematics,445,3,5.0,Ralph P Grimaldi,87.0
107,Higher Engineering Mathematics one,91,7,8.0,BV Ramana,65.0


In [34]:
import pickle
pickle.dump(rating_pivot,open('book.pkl','wb'))
pickle.dump(model_knn,open('model.pkl','wb'))

In [30]:
indices=pickle.load(open('book.pkl','rb'))
model= pickle.load(open('model.pkl','rb'))